In [ ]:
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
from torch.utils.data import DataLoader

tokenizer = T5Tokenizer.from_pretrained("panggi/t5-base-indonesian-summarization-cased")
model = T5ForConditionalGeneration.from_pretrained("panggi/t5-base-indonesian-summarization-cased")

dataset = load_dataset('csv', data_files='./data/summarize.csv', delimiter=';')

def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["input"]]
    outputs = examples["output"]
    
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(outputs, max_length=128, truncation=True, padding="max_length")

    if labels["input_ids"] is None:
        print("Error: Labels are None")
    
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

train_df = tokenized_dataset["train"].to_pandas()

train_data, test_data = train_test_split(train_df, test_size=0.2)

train_dataset = tokenized_dataset["train"].select(train_data.index.values)
test_dataset = tokenized_dataset["train"].select(test_data.index.values)

def to_torch_dataset(tokenized_data):
    class SummarizationDataset(torch.utils.data.Dataset):
        def __init__(self, tokenized_data):
            self.input_ids = tokenized_data["input_ids"]
            self.attention_mask = tokenized_data["attention_mask"]
            self.labels = tokenized_data["labels"]

        def __len__(self):
            return len(self.input_ids)

        def __getitem__(self, idx):
            return {
                'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
                'attention_mask': torch.tensor(self.attention_mask[idx], dtype=torch.long),
                'labels': torch.tensor(self.labels[idx], dtype=torch.long)
            }
    
    return SummarizationDataset(tokenized_data)

train_torch_dataset = to_torch_dataset(train_dataset)
eval_torch_dataset = to_torch_dataset(test_dataset)

batch_size = 8
train_loader = DataLoader(train_torch_dataset, batch_size=batch_size, shuffle=True)
eval_loader = DataLoader(eval_torch_dataset, batch_size=batch_size)

print(f"Train dataset: {len(train_data)} samples")
print(f"Test dataset: {len(test_data)} samples")

epochs = 3
learning_rate = 3e-5

from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_loader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

def train_epoch(model, data_loader, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    for batch in data_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        
        optimizer.zero_grad()

        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()
    
    avg_loss = total_loss / len(data_loader)
    return avg_loss

def eval_epoch(model, data_loader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in data_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            total_loss += loss.item()

    avg_loss = total_loss / len(data_loader)
    return avg_loss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    train_loss = train_epoch(model, train_loader, optimizer, scheduler, device)
    print(f"Train Loss: {train_loss}")

    eval_loss = eval_epoch(model, eval_loader, device)
    print(f"Eval Loss: {eval_loss}")

model.save_pretrained("./t5-finetuned-indonesian-summarization")
tokenizer.save_pretrained("./t5-finetuned-indonesian-summarization")

print("Fine-tuning selesai dan model disimpan.")


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/131 [00:00<?, ? examples/s]

Train dataset: 104 samples
Test dataset: 27 samples
Epoch 1/3


c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Train Loss: 3.6777638288644643
Eval Loss: 3.2946799993515015
Epoch 2/3


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("./summarization")
model = T5ForConditionalGeneration.from_pretrained("./summarization")

input_text = """
Pengenalan Android Apa itu Android Android adalah sistem operasi yang dikeluarkan oleh Google. Android dibuat khusus untuk smartphone dan tablet. Berbagai macam produsen telah menggunakan Android sebagai sistem operasi untuk peranti ( device ) yang mereka produksi. Android juga mempunyai  store  dengan lebih dari 2.5 miliar pengguna aktif per bulannya, per Mei 2019  [1] . Mengapa Android Kenapa menggunakan Android? Android memanjakan penggunanya dengan fitur yang sangat canggih dan tampilan yang bagus. Sistem Android dapat digunakan sebagai alat multimedia seperti pemutar musik dan video. Ia juga memiliki perangkat keras seperti  accelerometer ,  gyroscope  dan sensor lainnya. Di samping itu ada beberapa hal yang membuat Android menjadi sistem operasi yang memang layak digunakan oleh pengguna atau dikembangkan para developer, seperti yang akan diuraikan berikut ini. Sistem Operasi Smartphone Terpopuler Pada tahun 2013, Android menjadi  operation system  (OS) terlaris pada tablet dan smartphone. Kini  market share  Android sedikitnya 70 % dari total penjualan smartphone di tingkat global (statista.com)  [2] . Tercatat pada tahun 2016 Android  store  memiliki lebih dari 2.8 juta aplikasi  [3] . Android menarik bagi perusahaan teknologi yang membutuhkan barang siap jadi, biaya rendah dan kustomisasi OS untuk perangkat teknologi tinggi mereka. Hal ini menjadi daya tarik bagi banyak perusahaan, sehingga mereka memilih Android. Source code  dari Android bersifat  open source . Ini adalah hal menarik bagi komunitas developer, karena lisensi  open source  sangat mendukung untuk mengembangkan produknya dengan aman. Store Aplikasi Android bisa didistribusikan menggunakan  web ,  copy APK , dan  store . Android  s tore  , yaitu Google Play, merupakan cara termudah bagi para developer untuk mendistribusikan aplikasinya ke pasar dengan miliaran pengguna. Google play merupakan  store  resmi Android yang dikelola oleh Google. Pengguna bisa mencari dan mengunduh aplikasi yang dikembangkan dengan menggunakan Android Software Development Kit. Google Play tak hanya menawarkan aplikasi. Ada beragam konten lainnya yang dapat dinikmati pengguna, misalnya media digital, musik, buku, majalah, film dan program televisi. Bagaimana para developer memonetisasi aplikasi yang ada di dalam Google Play? Strategi monetisasi aplikasi yang ditawarkan Google Play ada bermacam-macam. Dimulai dari  app  berbayar ( paid distribution ), pembelian dalam aplikasi ( in-app purchase ), langganan ( subscriptions ), dan iklan ( ads ). Tentunya developer harus mengikuti aturan yang ada untuk memastikan bahwa pengguna mendapatkan pengalaman ( user experience)  terbaik. Development Kit untuk Developer Android Software Development Kit (SDK) merupakan  kit  yang bisa digunakan oleh para developer untuk mengembangkan aplikasi berbasis Android. Di dalamnya, terdapat beberapa  tools  seperti  debugger ,  software libraries ,  emulator , dokumentasi,  sample code  dan tutorial. Bahasa pemrograman yang sering digunakan untuk mengembangkan aplikasi Android adalah Java. Namun ada beberapa bahasa lainnya yang dapat digunakan, seperti C++ dan Go. Pada IO 2017 Google juga menetapkan Kotlin sebagai tambahan bahasa resmi  [4] . Berbicara tentang pemrograman tentunya tak lepas dari  Integrated Development Environment  (IDE). Pada 2014 Google mengeluarkan IDE yang bernama Android Studio yang berbasiskan Intellij IDEA. Dengan menggunakan Android Studio, para developer dapat membuat aplikasi dari nol hingga dipublikasikan ke dalam  store . Android Studio juga mempunyai beberapa fitur  built-in  yang sangat membantu para developer untuk memaksimalkan proses pembuatan aplikasi. Fitur-fitur ini misalnya project template, layout editor, debugging, testing, hingga membuat berkas APK. Sejarah Perkembangan Android Berikut adalah rangkaian sejarah perkembangan Android yang resmi diluncurkan oleh Google dari waktu ke waktu. Platform Version Version Code Release date API level Feature Highlight Icon 13 TIRAMISU August 15, 2022 33 Themed app icons Per-app language preferences Photo picker Notification permissions 12 S October 4, 2021 31-32 Material You Widget improvement Approximate location permissions Privacy Dashboard 11 R September 8, 2020 30 Chat Bubbles Screen Recorder Device Control Predictive Tool One-time permission  10 Q September 3, 2019 29 Live Caption Smart Reply Sound Amplifier Dark Theme Privacy & Security Digital Wellbeing 9 P August 6, 2018 28 Adaptive Battery Adaptive Brightness 8.0-8.1 O October 25, 2017 26 - 27 Picture-in-Picture 7.1 - 7.1.2 N August 22, 2016 24 - 25 Multi-window GIF Keyboard 6.0 - 6.0.1 M October 5, 2015 23 Now On Tap Permissions Battery (Doze & App Standby) 5.1 - 5.1.1 LOLLIPOP November 12, 2014 21 - 22 Material Design Multiscreen Notifications 4.4 - 4.4.4 KITKAT October 31, 2013 19 - 20 Voice : Ok Google Immersive Design Smart Dialer 4.1 - 4.3.1 JELLY_BEAN July 9, 2012 16-18 Google Now Actionable Notifications Account Switching 4.0 - 4.6 ICE_CREAM_SANDWICH October 19, 2011 14-15 Custom Home Screen Data Usage Control Android Beam 3.0 - 3.2.6 HONEYCOMB February 22, 2011 11 - 13 Tablet-Friendly Design System Bar Quick Settings 2.3 - 2.3.7 GINGERBREAD February 9, 2011 9 - 10 Gaming APIs NFC Battery Management 2.2 - 2.23 FROYO May 20, 2010 8 Voice Action Portable Hotspot Dalvik JIT 2.0 - 2.1 ECLAIR October 26, 2009 5-7 Google Maps Navigation Home Screen Customization Speech-to-Text 1.6 DONUT September 15, 2009 4 Quick Search Box Screen Size Diversity Android Market 1.5 CUPCAKE April 27, 2009 3 - Saat versi baru Android dirilis, developer hendaknya terus mengikuti  best-practice  terbaru untuk memastikan aplikasi yang dibuat tetap memberikan pengalaman yang terbaik di sebanyak mungkin device. Untuk mencari tahu tentang versi terbaru, Anda dapat melihatnya pada tautan berikut. What is API Level? ART dan DVM Dari tabel sejarah perkembangan di atas dapat kita lihat ada kolom DVM / ART. Kolom ini menunjukkan eksekusi kompilasi ketika menjalankan aplikasi Android. Pada API KitKat dan sebelumnya Android menggunakan DVM (Dalvik Virtual Machine).  DVM  menerapkan pendekatan JIT ( Just-In-Time ), di mana kompilasi dijalankan ketika ada permintaan untuk menjalankan aplikasi. Sedangkan  ART  ( Android Runtime ) menerapkan pendekatan berbeda yaitu AOT ( Ahead-Of-Time ). AOT melakukan kompilasi pada saat proses instalasi aplikasi. Dari versi Lollipop hingga sekarang, Android sepenuhnya mengadopsi ART. Mengapa demikian? DVM menggunakan JIT yang berarti kompilasi dilakukan setiap kali aplikasi dijalankan. Hal ini sangat mempengaruhi kecepatan respon aplikasi. Setiap kali kita menyentuh ikon aplikasi maka kompilasi akan dilakukan. Tentu proses ini menghabiskan CPU dan berimbas pada relatif lebih borosnya penggunaan baterai. Beda dengan DVM, ART melakukan proses kompilasi pada saat proses instalasi. Jadi setiap kali aplikasi dijalankan, sudah tidak ada lagi proses kompilasi. Hal ini meningkatkan performa dalam menjalankan aplikasi. Selain itu karena penggunaan sumber daya CPU bisa dikurangi, pemakaian baterai jadi lebih hemat. Akan tetapi ART membutuhkan  space  (ukuran berkas) yang lebih besar jika dibandingkan dengan DVM. Jika ingin mendalami proses  run-time  yang ada di Android, silakan klik tautan berikut ini:  Android Runtime (ART) and Dalvik  Beberapa bacaan dasar yang dapat menambah wawasan Anda, antara lain:  Apa Itu Android? Android Studio Android Studio adalah Lingkungan Pengembangan Terpadu -  Integrated Development Environment  (IDE) untuk pengembangan aplikasi Android, berdasarkan  IntelliJ IDEA  . Selain merupakan editor kode IntelliJ dan alat pengembang yang berdaya guna, Android Studio menawarkan fitur lebih demi meningkatkan produktifitas Anda saat membuat aplikasi Android, misalnya: Template : template memulai project maupun Activity tanpa harus membuatnya dari nol. Intelligent code editor : code completion yang memudahkan untuk menulis kode dengan cepat tanpa harus menuliskan secara lengkap. Selain itu, juga ada warning apabila terdapat kesalahan penulisan kode. Design tool : digunakan untuk mendesain aplikasi beserta melihat preview secara langsung sebelum dijalankan. Flexible build system : Android Studio menggunakan Gradle yang fleksibel untuk menciptakan build variant yang berbeda untuk berbagai device. Anda juga dapat menganalisa prosesnya secara mendetail. Emulator : menjalankan aplikasi tanpa harus menggunakan device Android. Debugging : memudahkan untuk mencari tahu masalah. Testing : menjalankan pengujian untuk memastikan semua kode aman sebelum rilis. Publish : membuat berkas AAB/APK dan menganalisanya guna dibagikan dan di-publish ke PlayStore. Dilengkapi dengan Instant Run untuk melihat perubahan tanpa harus build project dari awal. Integrasi : Terhubung dengan berbagai layanan yang memudahkan untuk mengembangkan aplikasi, seperti Github, Firebase, dan Google Cloud. Persyaratan Sistem Windows Mac Microsoft® Windows® 8/10 (64-bit) x86_64 CPU architecture; Intel Core 2nd Gen atau lebih, atau AMD CPU dengan support  Windows Hypervisor RAM 8 GB atau lebih Ruang disk minimum yang tersedia 8 GB (IDE + Android SDK + Android Emulator) Resolusi layar minimum 1280 x 800 MacOS® 10.14 (Mojave) atau lebih baru ARM-based chips, atau Intel Core 2nd Gen atau lebih dengan support  Hypervisor.Framework RAM 8 GB atau lebih Ruang disk minimum yang tersedia 8 GB (IDE + Android SDK + Android Emulator) Resolusi layar minimum 1280 x 800 Linux Chrome OS Linux 64-bit yang support GNOME, KDE, atau Unity DE, GNU C Library (glibc) 2.31 atau lebih x86_64 CPU architecture; Intel Core atau lebih 2nd Gen, atau AMD processor dengan support AMD Virtualization (AMD-V) dan SSSE3 RAM 8 GB atau lebih Ruang disk minimum yang tersedia 8 GB (IDE + Android SDK + Android Emulator) Resolusi layar minimum 1280 x 800 Disarankan 8 GB RAM atau lebih Ruang disk minimum yang tersedia 4 GB Resolusi layar minimum 1280 x 800 Disarankan Intel i5 atau lebih (U series atau lebih)  Perangkat yang direkomendasikan bisa dilihat di  chromeos.dev .
"""

input_ids = tokenizer.encode(input_text, return_tensors='pt')
summary_ids = model.generate(input_ids,
            max_length=500,
            min_length=300,
            num_beams=4,
            repetition_penalty=2.0, 
            length_penalty=0.8,
            early_stopping=False,
            no_repeat_ngram_size=3, 
            use_cache=True)

summary_text = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print(summary_text)



Android Studio merupakan Lingkungan Pengembangan Terpadu - Integrated Development Environment (IDE) untuk pengembangan aplikasi Android, berdasarkan Intellij IDEA. Selain merupakan editor kode IntelliJ dan alat pengembang yang berdaya guna, Android Studio menawarkan fitur lebih demi meningkatkan produktifitas Anda saat membuat aplikasi Android , misalnya : Template : template memulai project maupun Activity tanpa harus membuatnya dari nol; Intelligent code editor : code completion yang memudahkan untuk menulis kode dengan cepat tanpa harus menuliskan secara lengkap. Untuk mencari tahu tentang versi terbaru, sila klik tautan berikut di iReporter atau klik pautan di bawah untuk mengetahui apa itu Android? anda dapat melihat beberapa hal Android yang ada di dalamnya seperti : Google Play, Facebook, Youtube, Cinderella, Microsoft Word, dan Peta Google. [Google Play] adalah cara termudah bagi para developer untuk mendistribusikan aplikasinya ke pasar dengan miliaran pengguna. Klik link di s